In [ ]:
import pandas as pd
import re

In [ ]:
df_maindata = pd.read_excel(r"RAW DATA.xlsx")
df_proc_codeandname = pd.read_excel(r"PROCEDURE CODE DICTIONARY.xlsx")
df_insuranceplan = pd.read_excel(r"INSURANCE PLAN DICTIONARY.xlsx")

In [ ]:
rowcount_datamain = sum(1 for row in df_maindata["Visit ID"])
rowcount_procedure = sum(1 for row in df_proc_codeandname["Procedure Code"])
rowcount_insurance = sum(1 for row in df_insuranceplan["Insurance ID"])

In [ ]:
#data from the main file
main_visit_id = []
main_payer_plan = []
main_pat_name = []
main_visit_date = []
main_clinic_name = []
main_proc_code = []
main_total_adj_amount = []
main_patid = []


for i in range(0,rowcount_datamain):
    main_visit_id.append(str(df_maindata["Visit ID"][i]))
    
    main_payer_plan.append(str(df_maindata["Charge Payer Plan"][i]).lower())
    
    main_pat_name.append(str(df_maindata["Patient Full Name"][i]).lower())
       
    main_visit_date.append(str(df_maindata["Days in Visit Date"][i]).lower())   
        
    main_clinic_name.append(str(df_maindata["Clinic Name"][i]).lower())
    
    main_proc_code.append(str(df_maindata["Procedure Code"][i]).lower())
    
    main_total_adj_amount.append(str(df_maindata["Total Adjustment Amount"][i]).lower())
    
    main_patid.append(df_maindata["Patient ID"][i])

In [ ]:
# print(main_visit_id[0])
# print(main_visit_id[-1])

# print(main_payer_plan[0])
# print(main_payer_plan[-1])

# print(main_pat_name[0])
# print(main_pat_name[-1])

# print(main_visit_date[0])
# print(main_visit_date[-1])

# print(main_clinic_name[0])
# print(main_clinic_name[-1])

# print(main_proc_code[0])
# print(main_proc_code[-1])

# print(total_adj_amount[0])
# print(total_adj_amount[-1])

In [ ]:
#insurance data only the one that is required
insurance_dict = {}
for i in range(0,rowcount_insurance):
    insurance_plan = df_insuranceplan["Insurance Plan"][i].lower()
    if insurance_plan not in insurance_dict:
        address = str(df_insuranceplan["Address 1"][i]).lower()
        city = str(df_insuranceplan["City"][i]).lower()
        state = str(df_insuranceplan["State"][i]).lower()
        zipcode = str(df_insuranceplan["Zip"][i]).lower()
        phone = str(df_insuranceplan["Phone"][i]).lower()
        
        insurance_dict[insurance_plan] = [address, city, state, zipcode, phone]
    else:
        address = str(df_insuranceplan["Address 1"][i]).lower()
        city = str(df_insuranceplan["City"][i]).lower()
        state = str(df_insuranceplan["State"][i]).lower()
        zipcode = str(df_insuranceplan["Zip"][i]).lower()
        phone = str(df_insuranceplan["Phone"][i]).lower()
        
        val = insurance_dict[insurance_plan]
        val = [address, city,state, zipcode, phone]
        insurance_dict[insurance_plan] = val

In [ ]:
# print(insurance_dict)

In [ ]:
#procedure code and names
proc_dict = {}

for i in range(0,rowcount_procedure):
    code1 = df_proc_codeandname["Procedure Code"][i]
    if type(code1) == str:
        code = code1.lower()
    else:
        code = code1
    
    if code not in proc_dict:
        name = df_proc_codeandname["Procedure Name"][i].lower()
        proc_dict[code] = [name]
    else:
        name = df_proc_codeandname["Procedure Name"][i].lower()
        proc_dict[code].append(name)


In [ ]:
# print(proc_dict)

In [ ]:
frp_emp_name = []
frp_address = []
frp_city = []
frp_st = []
frp_zip = []
frp_phone = []


proc_desc = []

for i in range(0,rowcount_datamain):
    main_payer_plan_name = main_payer_plan[i]
    
    if main_payer_plan_name in insurance_dict:
        values = insurance_dict[main_payer_plan_name]
        
        frp_emp_name.append(main_payer_plan_name)
        frp_address.append(values[0])
        frp_city.append(values[1])
        frp_st.append(values[2].upper())
        frp_zip.append(values[3])
        frp_phone.append(values[4])
    else:
        values = [" "," "," "," "," "]
        
        frp_emp_name.append(main_payer_plan_name)
        frp_address.append(values[0])
        frp_city.append(values[1])
        frp_st.append(values[2].upper())
        frp_zip.append(values[3])
        frp_phone.append(values[4])
        
        
    proc_code1 = main_proc_code[i]
    ans = bool(proc_code1.isdigit())
    if ans == False:
        proc_code = proc_code1
    else:
        proc_code = int(proc_code1)
    if proc_code in proc_dict:
        value1 = proc_dict[proc_code]
        if len(value1) > 1:
            val = value1[-1]
        else:
            val = value1[0]
    else:
        val = " "
    proc_desc.append(val)

In [ ]:
df1 = pd.DataFrame(list(zip(main_visit_id, frp_emp_name,frp_address, frp_city, frp_st, frp_zip, frp_phone, main_pat_name, main_patid, main_visit_date, main_clinic_name, main_proc_code, proc_desc, main_total_adj_amount)), columns =["Visit ID", "FRP - Employer Name", "FRP Address 1", "FRP City", "FRP St", "FRP Zip", "FRP Phone", "Patient Full Name", "Patient ID", "Visit Date", "Location", "Procedure Code", "Procedure Description", "Total Adjustment Amount"])
df1.to_excel(r"OUTPUT FILE.xlsx")